In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install decord -q

In [ ]:
from transformers import Blip2ForConditionalGeneration, AutoProcessor
import torch
from PIL import Image
import os
import sys
import decord
from decord import VideoReader, cpu
from transformers import BitsAndBytesConfig
import numpy as np
# sys.path.append("/kaggle/input/eilev-git/EILEV-main/")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kimmin1253/testvideo-ive")

print("Path to dataset files:", path)

In [ ]:
# /kaggle/input/eilev-git/EILEV-main/eilev/
from eilev.model.v2 import VideoBlipForConditionalGeneration

In [ ]:
from accelerate import infer_auto_device_map

# ✅ 모델 로드 시 `device_map` 명시적으로 지정
device_map = infer_auto_device_map(model, max_memory={0: "8GB", 1: "8GB"}, no_split_module_classes=["language_model"])

# ✅ EILeV 모델 경로
model_name = "kpyu/eilev-blip2-opt-2.7b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ 모델 로드 (device_map 수정)
model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map= device_map, #{"": 0},  # 🔥 명확하게 GPU 0번에 로드
    quantization_config = BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype=torch.float16,  # 메모리 최적화
)
# ✅ `language_model`이 device_map에 포함되지 않으면 수동으로 분배
# model = dispatch_model(model, device_map="auto")
processor = AutoProcessor.from_pretrained(model_name)

In [ ]:
print(f"✅ Loaded Model Class: {type(model)}")

In [ ]:
# ✅ BLIP-2의 입력 처리 방식 적용
# ✅ 비디오 처리 및 입력 변환

from eilev.model.utils import process

# ✅ 비디오(mp4) 파일 경로
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"

# ✅ OpenCV를 사용한 비디오 로드
cap = cv2.VideoCapture(video_path)
frames = []
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = 16  # 초당 1프레임씩 총 16개 샘플링

# ✅ 프레임 인덱스 선택
indices = np.linspace(0, frame_count - 1, num_frames).astype(int)

for i in indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR → RGB 변환 필요
    frames.append(frame)

cap.release()  # 🔥 비디오 파일 닫기


# 프레임을 Tensor로 변환: 최종 shape: (1, 3, num_frames, 224, 224)
frames_tensor = torch.from_numpy(np.stack(frames)).permute(0, 3, 1, 2)  # (num_frames, 3, H, W)
frames_tensor = frames_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
frames_tensor = frames_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)

# --------------------------
# 3. 비디오 입력 전처리 (process 함수 사용)
# --------------------------
# 여기서 'text'는 나중에 프롬프트와 함께 입력될 텍스트 부분을 위한 것입니다.
# 하지만 우리는 직접 input_ids를 구성할 예정이므로 여기서는 비디오만 처리합니다.
inputs_video = process(processor, video=frames_tensor)


In [ ]:
# --------------------------
# 4. 입력 시퀀스와 video_input_mask 생성 (EILeV 방식)
# --------------------------
# tokenizer 가져오기
tokenizer = processor.tokenizer

# 프롬프트 구성: 비디오 토큰 자리와 텍스트 프롬프트를 모두 포함
# 예시 프롬프트: "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids


# 필요한 토큰: bos_token, video placeholders (num_query_tokens), newline, prompt tokens
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 구성:
#   [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # shape: (1, L)
num_tokens = input_ids.shape[1]

# video_input_mask: True for video token positions, False for others.
# bos 토큰: 0, video placeholders: 1, newline + prompt: 0
video_input_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
video_input_mask = torch.tensor([video_input_mask], dtype=torch.bool).to(device)



In [ ]:
# --------------------------
# 5. Combine 입력 (비디오 픽셀 값)
# --------------------------
# inputs_video는 딕셔너리로, 특히 "pixel_values"가 포함됨.
inputs = {}
inputs["input_ids"] = input_ids
inputs["pixel_values"] = inputs_video["pixel_values"].to(device)

# 디버깅 프린트
print(f"🔍 input_ids.shape: {inputs['input_ids'].shape}")      # 예: (1, L) where L = 1+32+1+len(prompt_tokens)
print(f"🔍 video_input_mask.shape: {video_input_mask.shape}")    # (1, L)
print(f"🔍 video_input_mask sum (should be {num_query_tokens}): {video_input_mask.sum().item()}")
print(f"🔍 pixel_values.shape: {inputs['pixel_values'].shape}")    # (1, 3, num_frames, 224, 224)

In [ ]:
# ✅ GPU 이동
device = model.device
inputs = {key: val.to(device) for key, val in inputs.items()}
video_input_mask = video_input_mask.to(device)

In [ ]:
# ✅ EILeV 모델 실행
with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        video_input_mask=video_input_mask,  # 📌 필수 입력 추가
        max_new_tokens=30,
        num_beams=4,
        repetition_penalty=1.5,
    )

# ✅ 생성된 텍스트 디코딩
generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

# ✅ 결과 출력
print("\n🎬 Video Description Generated:")
print(generated_text)

In [ ]:
# ✅ 기존 방식: 전체 프레임을 통합하여 하나의 설명 생성
with torch.no_grad():  # 🔥 그래디언트 계산 비활성화 (메모리 최적화)
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=pixel_values,  # 🔥 5D 텐서로 변환된 비디오 입력
        video_input_mask=video_input_mask,  # ✅ 필수 추가
        max_new_tokens=80,  # 🔥 설명 길이 조정
        num_beams=4,  # 🔥 빔 서치 활용
        repetition_penalty=1.3,  # 🔥 반복 줄이기
    )

# ✅ 생성된 텍스트 디코딩
generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

print("\n🎬 Video Description Generated:")
print(generated_text)

In [ ]:
frame_paths = [
    "/kaggle/input/test-img-ive/Ady0zhVZn58_frame_0000.jpg",
    "/kaggle/input/test-img-ive/Ady0zhVZn58_frame_0029.jpg",
    "/kaggle/input/test-img-ive/Ady0zhVZn58_frame_0058.jpg",
    "/kaggle/input/test-img-ive/Ady0zhVZn58_frame_0087.jpg",
    "/kaggle/input/test-img-ive/Ady0zhVZn58_frame_0116.jpg",
]
frames = [Image.open(path).convert("RGB") for path in frame_paths]

In [ ]:
# ✅ BLIP-2의 새로운 입력 처리 방식 적용
prompt = "Question: Describe in detail what actions are taking place in the video.\nAnswer:"

inputs = processor(
    images=frames,
    text=[prompt] * len(frames),  # 🔥 모든 프레임에 같은 프롬프트 적용
    return_tensors="pt",
    padding=True,  # 🔥 패딩 추가하여 크기 맞추기
)

# 🚀 Debugging: 변환된 입력 데이터 크기 확인
print(f"Input IDs shape: {inputs['input_ids'].shape}")  # (batch_size, sequence_length)
print(f"Pixel Values shape: {inputs['pixel_values'].shape}")  # (batch_size, 3, 224, 224)
# ✅ GPU 이동
inputs = {key: val.to("cuda") for key, val in inputs.items()}

print("✅ 입력 데이터 변환 완료!")

In [ ]:
# ✅ 기존 방식: 전체 프레임을 통합하여 하나의 설명 생성
with torch.no_grad():  # 🔥 그래디언트 계산 비활성화 (메모리 최적화)
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=80,  # 길이 조정 유지
        num_beams=4,  # 적절한 빔 서치 유지
        repetition_penalty=1.3,  # 🔥 반복 문장 줄이기
    )

# ✅ 생성된 텍스트 디코딩
generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

In [ ]:
##################################################################################

In [ ]:
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image

# ==================================================
# 1. 모델 및 프로세서 로드 (EILeV 전용)
# ==================================================
model_name = "kpyu/eilev-blip2-opt-2.7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# EILeV 전용 클래스로 로드 (8-bit, GPU 자동 배분)
model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)
# 모델은 이미 올바른 디바이스에 로드됨 (8-bit 양자화 시 .to(device) 불필요)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

# ==================================================
# 2. 비디오(mp4) 파일에서 프레임 추출 (OpenCV 사용)
# ==================================================
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
frames = []
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = 16  # 예: 초당 1프레임씩 총 16개 샘플링



# TODO
=====================================================
초당 2 프레임씩 num_frames= 10 (5초) 짜리 frames 생성

total_frames_list = []

10_frames_list = 5초씩 자르기

for 24번 동안:
total_frames_list.append(10_frames_list)


results = []
#EILeV 모델 24번 호출:                         
for 10_frames_list in total_frames_list:
    10_frames_list를 넣기
    result = {}
    blip_result = videoblipmodel(input_is, mask, )

    result['time'] = time 정보
    result['caption'] = blip_result
    
    results.append(result)

results 파일로 timeframe과 함께 파일로 쓰기
=====================================================



indices = np.linspace(0, frame_count - 1, num=num_frames).astype(int)
for i in indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
    # OpenCV는 BGR → RGB 변환 필요
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
cap.release()

# 프레임을 Tensor로 변환 (최종 shape: [1, 3, num_frames, 224, 224])
# (num_frames, H, W, C) → (1, C, num_frames, H, W)
frames_tensor = torch.from_numpy(np.stack(frames)).permute(0, 3, 1, 2)  # (num_frames, 3, H, W)
frames_tensor = frames_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
frames_tensor = frames_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)

# ==================================================
# 3. 비디오 입력 전처리 (process() 함수 사용)
# ==================================================
# 여기서는 비디오 부분만 처리하여 pixel_values를 생성합니다.
inputs_video = process(processor, video=frames_tensor, text=None)
# inputs_video["pixel_values"]의 shape: (1, 3, num_frames, 224, 224)



In [ ]:
# ==================================================
# 4. 입력 시퀀스 및 video_input_mask 구성 (EILeV 형식)
# ==================================================
# EILeV는 입력 시퀀스에 비디오 관련 자리(쿼리 토큰)를 먼저 배치한 후, 프롬프트 텍스트를 추가합니다.
prompt = "Question: Describe in detail what actions are taking place in the video.\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 입력 시퀀스 구성: [bos] + [pad] * num_query_tokens + [newline] + prompt_tokens
input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # shape: (1, L)
num_tokens = input_ids.shape[1]

# video_input_mask: bos 토큰 → 0, 비디오 자리 (쿼리 토큰) → 1, 나머지(텍스트) → 0
video_input_mask_list = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
video_input_mask = torch.tensor([video_input_mask_list], dtype=torch.bool).to(device)

# ==================================================
# 5. 최종 입력 구성
# ==================================================
inputs = {}
inputs["input_ids"] = input_ids
inputs["pixel_values"] = inputs_video["pixel_values"].to(device)

# ==================================================
# 6. 디버깅 프린트
# ==================================================
print(f"🔍 input_ids.shape: {inputs['input_ids'].shape}")          # 예: (1, L) where L = 1 + 32 + 1 + len(prompt_tokens)
print(f"🔍 pixel_values.shape: {inputs['pixel_values'].shape}")        # (1, 3, num_frames, 224, 224)
print(f"🔍 video_input_mask.shape: {video_input_mask.shape}")          # (1, L)
print(f"🔍 video_input_mask sum (should be {num_query_tokens}): {video_input_mask.sum().item()}")

# ==================================================
# 7. 모델 실행 (EILeV)
# ==================================================
with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        video_input_mask=video_input_mask,  # 필수 입력: shape (1, L)
        max_new_tokens=80,
        num_beams=4,
        repetition_penalty=1.3,
    )

# ==================================================
# 8. 생성된 텍스트 디코딩 및 출력
# ==================================================
generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
print("\n🎬 Video Description Generated:")
print(generated_text)


In [ ]:
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image

# ==================================================
# 1. 모델 및 프로세서 로드 (EILeV 전용)
# ==================================================
model_name = "kpyu/eilev-blip2-opt-2.7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# EILeV 전용 모델 로드 (8-bit, GPU 자동 배분)
model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

# ==================================================
# 2. 비디오 전체 정보 가져오기
# ==================================================
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

# ==================================================
# 3. 세그먼트 분할 및 캡션 생성
# ==================================================
segment_duration = 5  # 세그먼트 길이 (초)
frames_per_segment = int(fps * segment_duration)

metadata = []  # 각 세그먼트의 메타데이터를 저장할 리스트

for start_frame in range(0, total_frames, frames_per_segment):
    # 세그먼트의 시작과 끝 계산
    end_frame = min(start_frame + frames_per_segment, total_frames)
    num_segment_frames = end_frame - start_frame
    
    # 세그먼트 내에서 균등하게 프레임 선택
    cap = cv2.VideoCapture(video_path)
    indices = np.linspace(start_frame, end_frame - 1, num=num_segment_frames).astype(int)
    segment_frames = []
    for i in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        segment_frames.append(frame)
    cap.release()
    
    if len(segment_frames) == 0:
        continue
    
    # -------------------------------
    # 3-1. 프레임 전처리: Tensor 변환
    # -------------------------------
    # (num_frames, H, W, C) → (1, 3, num_frames, H, W)
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    # -------------------------------
    # 3-2. 비디오 전처리: process() 함수 사용
    # -------------------------------
    inputs_video = process(processor, video=segment_tensor, text=None)
    
    # -------------------------------
    # 3-3. 입력 시퀀스 및 video_input_mask 구성
    # -------------------------------
    # 프롬프트 구성: 단일 Q&A 쌍
    prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
    prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids
    
    bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
    pad_token = tokenizer.pad_token_id
    newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
    num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수
    
    # 입력 시퀀스: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
    input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)
    num_tokens = input_ids.shape[1]
    
    # video_input_mask: bos → 0, 비디오 자리 (쿼리 토큰) → 1, 나머지 (텍스트) → 0
    video_input_mask_list = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
    video_input_mask = torch.tensor([video_input_mask_list], dtype=torch.bool).to(device)
    
    # -------------------------------
    # 3-4. 최종 입력 구성
    # -------------------------------
    inputs = {
        "input_ids": input_ids,
        "pixel_values": inputs_video["pixel_values"].to(device)
    }
    
    # 디버깅 프린트
    print(f"🔍 Segment starting at {start_frame/fps:.1f}s:")
    print(f"   input_ids.shape: {inputs['input_ids'].shape}")
    print(f"   pixel_values.shape: {inputs['pixel_values'].shape}")
    print(f"   video_input_mask.shape: {video_input_mask.shape}")
    print(f"   video_input_mask sum: {video_input_mask.sum().item()}")
    
    # -------------------------------
    # 3-5. 모델 실행 및 캡션 생성
    # -------------------------------
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            video_input_mask=video_input_mask,
            max_new_tokens=80,
            num_beams=4,
            repetition_penalty=1.3,
        )
    caption = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    # 세그먼트 시작 시간과 캡션 저장
    metadata.append({
        "start_time": start_frame / fps,
        "end_time": end_frame / fps,
        "caption": caption
    })

# ==================================================
# 4. 결과 출력: 각 세그먼트별 캡션
# ==================================================
for data in metadata:
    print(f"Segment {data['start_time']:.1f}s - {data['end_time']:.1f}s: {data['caption']}")


===========================비디오 5초마다 설명생성===============================
kpyu/eilev-blip2-opt-2.7b적용


In [ ]:
!pip install -U bitsandbytes


In [ ]:
import sys
sys.path.append("/kaggle/input/eilev-git/EILEV-main")


In [ ]:
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image
import json


In [ ]:
# ==================================================
# 1. 모델 및 프로세서 로드 (EILeV 전용)
# ==================================================
model_name = "kpyu/eilev-blip2-opt-2.7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True)


In [ ]:
# EILeV 전용 모델 로드 (8-bit, GPU 자동 배분)
model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)
# 모델은 이미 올바른 디바이스에 로드됨 (8-bit 양자화 시 .to(device) 불필요)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

In [ ]:
# ==================================================
# 2. 비디오 전체 정보 가져오기
# ==================================================
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
print(f"Total frames: {total_frames}, FPS: {fps}")

In [ ]:
# ==================================================
# 3. 세그먼트 분할 및 캡션 생성
# ==================================================
segment_duration = 5  # 세그먼트 길이 (초)
frames_per_segment = int(fps * segment_duration)

# 프롬프트 및 입력 시퀀스 구성 (모든 세그먼트에 동일)
prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 입력 시퀀스: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

# video_input_mask: bos 토큰 → 0, video 자리 (쿼리 토큰) → 1, 나머지 (텍스트) → 0
base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# 결과 저장 리스트
metadata = []


In [ ]:
# ==================================================
# 4. 세그먼트별 캡션 생성
# ==================================================
cap = cv2.VideoCapture(video_path)
for start_frame in range(0, total_frames, frames_per_segment):
    end_frame = min(start_frame + frames_per_segment, total_frames)
    # 세그먼트 프레임 추출
    segment_frames = []
    for i in range(start_frame, end_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR → RGB
        segment_frames.append(frame)
    if len(segment_frames) == 0:
        continue
    
    # 프레임을 Tensor로 변환: (num_frames, H, W, C) → (1, 3, num_frames, H, W)
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    # EILeV 공식 process() 함수로 비디오 입력 전처리 (시간 정보 유지)
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)  # (1, 3, num_frames, 224, 224)
    
    # 최종 입력 구성: 동일한 텍스트 입력 (base_input_ids)와 base_video_mask 사용
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask  # shape: (1, num_tokens)
    
    # 디버깅 프린트 (각 세그먼트별)
    print(f"🔍 Segment starting at {start_frame/fps:.1f}s:")
    print(f"   input_ids.shape: {inputs_seg['input_ids'].shape}")    # (1, num_tokens)
    print(f"   pixel_values.shape: {inputs_seg['pixel_values'].shape}")  # (1, 3, num_frames, 224, 224)
    print(f"   video_input_mask.shape: {video_input_mask_seg.shape}")  # (1, num_tokens)
    print(f"   video_input_mask sum (should be {num_query_tokens}): {video_input_mask_seg.sum().item()}")
    # 모델 실행: 세그먼트별 캡션 생성
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=80,
            num_beams=4,
            repetition_penalty=1.3,
        )
    caption = processor.batch_decode(output, skip_special_tokens=True)[0]
    
    # 세그먼트 시작 시간, 종료 시간, 캡션 저장
    metadata.append({
        "start_time": start_frame / fps,
        "end_time": end_frame / fps,
        "caption": caption
    })
cap.release()

In [ ]:
# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['start_time']:.1f}s - {seg['end_time']:.1f}s: {seg['caption']}")

# Optional: 저장 (JSON 파일)
with open("video_captions.json", "w") as f:
    json.dump(metadata, f, indent=4)

===========================비디오 5초마다 설명생성=============================== kpyu/eilev-kpyu/eilev-blip2-flan-t5-xl적용

In [ ]:
import sys
sys.path.append("/kaggle/input/eilev-git/EILEV-main")

import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image

In [ ]:
# 1. 모델 및 프로세서 로드 (FLAN‑T5 기반 EILeV)
model_name = "kpyu/eilev-blip2-flan-t5-xl"
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True)

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

In [ ]:
# 2. 비디오에서 프레임 추출 (예: 첫 5초, 초당 1프레임)
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Total frames: {frame_count}, FPS: {fps}")
num_frames = int(fps * 5)
frames = []
indices = np.linspace(0, num_frames - 1, num=num_frames).astype(int)
for i in indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
cap.release()

In [ ]:
# 3. 프레임 전처리 (텐서 변환 및 재배열)
frames_tensor = torch.from_numpy(np.stack(frames)).permute(0, 3, 1, 2)  # (num_frames, 3, H, W)
frames_tensor = frames_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
frames_tensor = frames_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)

inputs_video = process(processor, video=frames_tensor, text=None)


In [ ]:
# 4. 입력 시퀀스 및 video_input_mask 구성 (학습 시 사용된 Q&A 형식 유지)
prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

input_ids_list = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
input_ids = torch.tensor([input_ids_list], dtype=torch.long).to(device)
num_tokens = input_ids.shape[1]
video_input_mask_list = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
video_input_mask = torch.tensor([video_input_mask_list], dtype=torch.bool).to(device)

inputs = {
    "input_ids": input_ids,
    "pixel_values": inputs_video["pixel_values"].to(device)
}

# 디버깅: 입력 데이터 크기 확인
print(f"🔍 input_ids.shape: {inputs['input_ids'].shape}")         # (1, L)
print(f"🔍 pixel_values.shape: {inputs['pixel_values'].shape}")       # (1, 3, num_frames, 224, 224)
print(f"🔍 video_input_mask.shape: {video_input_mask.shape}")         # (1, L)
print(f"🔍 video_input_mask sum (should be {num_query_tokens}): {video_input_mask.sum().item()}")


In [ ]:
# 5. 모델 실행 및 캡션 생성 (디코딩 파라미터 조정)
with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        video_input_mask=video_input_mask,
        max_new_tokens=100,      # 더 긴 답변을 위해 늘림
        num_beams=6,             # 빔 서치 증가
        temperature=0.7,         # 온도 조절로 다양성 확보
        repetition_penalty=1.1,  # 반복 억제 완화
    )

generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
# 후처리: 첫 번째 Q&A 쌍만 추출
final_caption = generated_text.split("\n")[0]

print("\n🎬 Video Description Generated:")
print(final_caption)

In [ ]:
# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['start_time']:.1f}s - {seg['end_time']:.1f}s: {seg['caption']}")

# Optional: 저장 (JSON 파일)
with open("video_captions(flan-t5).json", "w") as f:
    json.dump(metadata, f, indent=4)

In [ ]:
import sys
sys.path.append("/kaggle/input/eilev-git/EILEV-main")

import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image

# ==================================================
# 1. 모델 및 프로세서 로드 (FLAN‑T5 기반 EILeV)
# ==================================================
model_name = "kpyu/eilev-blip2-flan-t5-xl"  # FLAN‑T5 기반 모델
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True)

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)
# 모델은 이미 적절한 디바이스에 로드됨 (추가 .to(device)는 필요없음)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

# ==================================================
# 2. 비디오(mp4) 파일에서 프레임 추출 (예: 첫 5초, 초당 1프레임)
# ==================================================
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Total frames: {frame_count}, FPS: {fps}")
num_frames = int(fps * 5)  # 5초 분량
frames = []
indices = np.linspace(0, num_frames - 1, num=num_frames).astype(int)
for i in indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
cap.release()

# ==================================================
# 3. 프레임 전처리: 텐서 변환 및 재배열 (최종 shape: [1, 3, num_frames, 224, 224])
# ==================================================
frames_tensor = torch.from_numpy(np.stack(frames)).permute(0, 3, 1, 2)  # (num_frames, 3, H, W)
frames_tensor = frames_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
frames_tensor = frames_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)

# ==================================================
# 4. 비디오 입력 전처리: process() 함수 사용
# ==================================================
inputs_video = process(processor, video=frames_tensor, text=None)

# ==================================================
# 5. 입력 시퀀스 및 video_input_mask 구성 (Q&A 형식 유지)
# ==================================================
prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32

# 전체 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
input_ids_list = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
input_ids = torch.tensor([input_ids_list], dtype=torch.long).to(device)
num_tokens = input_ids.shape[1]

# video_input_mask: [bos] → False, 다음 num_query_tokens → True, 나머지 → False
video_input_mask_list = [False] + [True] * num_query_tokens + [False] * (num_tokens - (1 + num_query_tokens))
video_input_mask = torch.tensor([video_input_mask_list], dtype=torch.bool).to(device)

# ==================================================
# 6. 최종 입력 구성
# ==================================================
inputs = {
    "input_ids": input_ids,
    "pixel_values": inputs_video["pixel_values"].to(device)
}

# 디버깅: 입력 데이터 크기 확인
print(f"🔍 input_ids.shape: {inputs['input_ids'].shape}")         # 예: (1, L)
print(f"🔍 pixel_values.shape: {inputs['pixel_values'].shape}")       # (1, 3, num_frames, 224, 224)
print(f"🔍 video_input_mask.shape: {video_input_mask.shape}")         # (1, L)
print(f"🔍 video_input_mask sum (should be {num_query_tokens}): {video_input_mask.sum().item()}")

# ==================================================
# 7. 모델 실행 (캡션 생성) - 디코딩 파라미터 조정
# ==================================================
with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        video_input_mask=video_input_mask,
        max_new_tokens=100,      # 더 긴 답변 생성
        num_beams=6,             # 빔 서치 확대
        temperature=0.7,         # 온도 조절
        repetition_penalty=1.1,  # 반복 억제 완화
    )

generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]

# 후처리: 첫 번째 Q&A 쌍만 사용 (필요에 따라 수정)
final_caption = generated_text.split("\n")[0]

print("\n🎬 Video Description Generated:")
print(final_caption)


blip2-plant-t5 사용 EILEV 내부함수 사용


In [ ]:
!pip install -U bitsandbytes 
!pip install pytorchvideo 

In [ ]:
import sys
# 캐글 입력 경로: EILeV 저장소가 /kaggle/input/eilev-git/EILEV-main 에 있다고 가정
sys.path.append("/kaggle/input/eilev-git/EILEV-main")

import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from eilev.data.utils import generate_input_ids_and_labels_from_interleaved  # 내부 입력 구성 함수 (옵션)
from PIL import Image
import json
import time
import re

In [ ]:
# ==================================================
# 1. 모델 및 프로세서 로드 (EILeV, OPT 기반)
# ==================================================
model_name = "kpyu/eilev-blip2-flan-t5-xl"  # 캐글에 업로드된 EILeV OPT 기반 체크포인트
device = "cuda" if torch.cuda.is_available() else "cpu"
# CPU/GPU 환경에 따라 quantization 사용 여부 결정 (캐글에서는 보통 GPU 사용)
quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
# GPU 환경에서 8-bit 양자화된 모델은 이미 올바른 디바이스에 로드되므로 .to(device) 불필요
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

In [ ]:
# ==================================================
# 2. 비디오 전체 정보 가져오기
# ==================================================
video_path = "/kaggle/input/testvideo-ive/Ady0zhVZn58.mp4"
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
print(f"Total frames: {total_frames}, FPS: {fps}")

In [ ]:
# # ==================================================
# # 3. 기본 입력 시퀀스 및 video_input_mask 구성
# #    (EILeV 학습 시 사용한 Q&A 형식 유지)
# # ==================================================
# prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
# prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

# bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
# pad_token = tokenizer.pad_token_id
# newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
# num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# # 전체 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
# base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
# base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
# num_tokens = base_input_ids.shape[1]

# # video_input_mask: bos 토큰 → 0, 다음 num_query_tokens (비디오 자리) → 1, 나머지 (텍스트 부분) → 0
# base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
# base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)


In [ ]:
# #opt 기반으로 설명생성 질문과 답은 후처리로 제거
# # ==================================================
# # 4. 세그먼트 분할 및 캡션 생성
# # ==================================================
# segment_duration = 5  # 세그먼트 길이 (초)
# frames_per_segment = int(fps * segment_duration)
# metadata = []  # 세그먼트별 메타데이터 저장 리스트

# cap = cv2.VideoCapture(video_path)
# current_segment = 0
# start_time_overall = time.time()

# while True:
#     start_frame = current_segment * frames_per_segment
#     if start_frame >= total_frames:
#         break
#     end_frame = min(start_frame + frames_per_segment, total_frames)
#     segment_frames = []
    
#     # 세그먼트 내 프레임 추출
#     for i in range(start_frame, end_frame):
#         cap.set(cv2.CAP_PROP_POS_FRAMES, i)
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         segment_frames.append(frame)
    
#     if len(segment_frames) == 0:
#         break
    
#     # 프레임 텐서 변환: (num_frames, H, W, C) → (1, 3, num_frames, H, W)
#     segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
#     segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
#     segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
#     # EILeV 공식 process() 함수 사용 (시간 정보 유지)
#     inputs_video = process(processor, video=segment_tensor, text=None)
#     pixel_values = inputs_video["pixel_values"].to(device)  # (1, 3, num_segment_frames, 224, 224)
    
#     # 최종 입력 구성: 동일한 텍스트 입력과 video mask 사용
#     inputs_seg = {
#         "input_ids": base_input_ids,
#         "pixel_values": pixel_values
#     }
#     video_input_mask_seg = base_video_mask  # (1, num_tokens)
    
#     seg_start_time = start_frame / fps
#     seg_end_time = end_frame / fps
#     print(f"Segment {current_segment} ({seg_start_time:.1f}s - {seg_end_time:.1f}s):")
#     print(f"   input_ids.shape: {inputs_seg['input_ids'].shape}")    # (1, num_tokens)
#     print(f"   pixel_values.shape: {inputs_seg['pixel_values'].shape}")  # (1, 3, num_segment_frames, 224, 224)
#     print(f"   video_input_mask.shape: {video_input_mask_seg.shape}")  # (1, num_tokens)
#     print(f"   video_input_mask sum (should be {num_query_tokens}): {video_input_mask_seg.sum().item()}")
    
#     # 모델 실행: 세그먼트별 캡션 생성
#     with torch.no_grad():
#         output = model.generate(
#             input_ids=inputs_seg["input_ids"],
#             pixel_values=inputs_seg["pixel_values"],
#             video_input_mask=video_input_mask_seg,
#             max_new_tokens=80,
#             num_beams=4,
#             repetition_penalty=1.3,
#         )
#     caption = processor.batch_decode(output, skip_special_tokens=True)[0]
#     print(f"   Generated caption: {caption}")
    
#     metadata.append({
#         "segment": current_segment,
#         "start_time": seg_start_time,
#         "end_time": seg_end_time,
#         "caption": caption
#     })
#     current_segment += 1

# cap.release()
# total_time = time.time() - start_time_overall
# print(f"Total segmentation and caption generation time: {total_time:.1f}s")
# with open("/kaggle/working/video_captions_opt.json", "w", encoding="utf-8") as f:
#     json.dump(metadata, f, indent=4, ensure_ascii=False)
# print("Result saved to 'output/video_captions_opt.json'")

In [ ]:
# ==================================================
# 3. 세그먼트 분할 및 캡션 생성
# ==================================================
segment_duration = 5  # 세그먼트 길이 (초)
frames_per_segment = int(fps * segment_duration)
metadata = []  # 각 세그먼트 메타데이터 저장 리스트

# FLAN‑T5에 맞게 프롬프트 변경: 단일 서술형 프롬프트 사용
# (Q&A 형식이 아니라 전체 설명 서술을 유도)
prompt = "Describe in detail what actions are taking place in the video."
# prompt_tokens는 내부 함수에서 자동 처리되도록 텍스트 입력을 None으로 처리합니다.
# (따라서 텍스트 프롬프트는 process() 함수 대신 별도로 input_ids를 구성하지 않습니다.)

# 대신, EILeV 내부 함수에서 비디오 부분만 처리하도록 합니다.
# 아래 base_input_ids와 video_input_mask는 EILeV 학습 시 사용한 형식으로 유지합니다.
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 기본 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
# 여기서는 prompt_tokens 대신 FLAN‑T5에 맞게 간단한 서술형 프롬프트를 사용할 수 있습니다.
# 예를 들어, 프롬프트에 "Describe in detail what is happening in the video." 만 사용합니다.
base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token]
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

# video_input_mask: bos 토큰 → 0, 다음 num_query_tokens (비디오 자리) → 1, 나머지는 0
base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

In [ ]:
# ==================================================
# 4. 세그먼트 분할 및 캡션 생성 (오버랩 적용)
# ==================================================
# 세그먼트 길이 및 오버랩 설정 (초 단위)
segment_duration = 5          # 전체 세그먼트 길이 (초)
overlap_duration = 2          # 오버랩 길이 (초)

frames_per_segment = int(fps * segment_duration)
overlap_frames = int(fps * overlap_duration)
# 각 세그먼트 시작 프레임: 세그먼트 전체 길이에서 오버랩 부분을 뺀 값만큼씩 진행
step_frames = frames_per_segment - overlap_frames

metadata = []  # 각 세그먼트의 메타데이터 저장 리스트
current_segment = 0
start_time_overall = time.time()

cap = cv2.VideoCapture(video_path)
while True:
    start_frame = current_segment * step_frames
    if start_frame >= total_frames:
        break
    # 각 세그먼트는 start_frame부터 frames_per_segment 만큼의 프레임을 포함
    end_frame = min(start_frame + frames_per_segment, total_frames)
    
    # 세그먼트에 해당하는 프레임 추출
    segment_frames = []
    for i in range(start_frame, end_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        segment_frames.append(frame)
    
    if len(segment_frames) == 0:
        break

    # 프레임을 텐서로 변환: (num_frames, H, W, C) → (1, 3, num_frames, H, W)
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    # EILeV 공식 process() 함수 사용 (text 입력은 None으로 설정)
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)
    
    # 최종 입력 구성: base_input_ids와 base_video_mask 사용
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask
    
    seg_start_time = start_frame / fps
    seg_end_time = end_frame / fps
    print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
    print(f"   input_ids.shape: {inputs_seg['input_ids'].shape}")
    print(f"   pixel_values.shape: {inputs_seg['pixel_values'].shape}")
    print(f"   video_input_mask.shape: {video_input_mask_seg.shape}")
    print(f"   video_input_mask sum (should be {num_query_tokens}): {video_input_mask_seg.sum().item()}")
    
    # 모델 실행: 캡션 생성
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=120,   # max_new_tokens 값을 늘려서 더 긴 캡션 생성
            num_beams=6,
            repetition_penalty=1.2,
        )
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
    caption = generated_text.strip()
    print(f"   Generated caption: {caption}")
    
    metadata.append({
        "segment": current_segment,
        "start_time": seg_start_time,
        "end_time": seg_end_time,
        "caption": caption
    })
    
    current_segment += 1

cap.release()
total_time = time.time() - start_time_overall
print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

with open("/kaggle/working/video_captions_flan_t5.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)
print("Result saved to '/kaggle/working/video_captions_flan_t5.json'")

In [1]:
!pip install -U bitsandbytes 
!pip install pytorchvideo 
!pip install decord imageio imageio-ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 45.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=ccbe3d561d2756f7eecce89db960d19c225ac9ccb301521c8ddc9fbd09fe49b0
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=5b77018f12e696091fce374801958c847b417a8ca01969a8126539dcfe7e3117
  Stored in d

In [7]:
import sys
# 캐글 입력 경로: EILeV 저장소가 /kaggle/input/eilev-git/EILEV-main 에 있다고 가정
sys.path.append("/kaggle/input/eilev-git/EILEV-main")


from collections import deque  # 슬라이딩 윈도우를 위해 필요합니다.
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image
import json
import time
import re
import imageio

In [3]:

# ==================================================
# 1. 모델 및 프로세서 로드 (EILeV, OPT 기반)
# ==================================================
model_name = "kpyu/eilev-blip2-flan-t5-xl"  # 캐글에 업로드된 EILeV OPT 기반 체크포인트
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/128k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
# ==================================================
# 2. 영상 읽기 (imageio를 스트리밍 방식으로 사용)
# ==================================================
video_path = "/kaggle/input/test-video-njz/FuJ1RiLoq-M.mp4"  # 새 영상 경로

try:
    reader = imageio.get_reader(video_path, 'ffmpeg')
    meta = reader.get_meta_data()
    fps = meta['fps']
    print(f"Imageio - FPS: {fps}")
except Exception as e:
    raise Exception(f"Failed to open video with imageio: {e}")

# ==================================================
# 3. 기본 입력 구성 (FLAN‑T5 단일 서술형 프롬프트 방식)
# ==================================================
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 추가할 프롬프트 텍스트 정의 (원하는 설명을 유도할 문구)
prompt = "Describe in detail what actions are taking place in the video, including dynamic movements like walking, running, or interacting with objects."
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

# 기본 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# ==================================================
# 4. 슬라이딩 윈도우를 이용한 세그먼트 분할 및 캡션 생성
# ==================================================
segment_duration = 5    # 세그먼트 길이 (초)
overlap_duration = 2    # 오버랩 길이 (초)

frames_per_segment = int(fps * segment_duration)
overlap_frames = int(fps * overlap_duration)
step_frames = frames_per_segment - overlap_frames

print(f"frames_per_segment: {frames_per_segment}, step_frames: {step_frames}")

metadata = []
current_segment = 0
start_time_overall = time.time()

# deque를 사용하여 sliding window 구성 (최대 frames_per_segment 개의 프레임 유지)
window = deque(maxlen=frames_per_segment)
frame_idx = 0

# 초기 윈도우 채우기
print("Preloading initial window...")
for i in range(frames_per_segment):
    try:
        frame = reader.get_data(i)
        window.append(frame)
        frame_idx = i + 1
    except Exception:
        break

if len(window) < frames_per_segment:
    print("Not enough frames to form a full segment. Exiting.")
else:
    # 첫 세그먼트 처리 (프레임 0 ~ frames_per_segment-1)
    seg_start_time = 0.0
    seg_end_time = frames_per_segment / fps
    segment_frames = list(window)
    
    # 프레임 텐서 변환: (num_frames, H, W, C) → (1, 3, num_frames, H, W)
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)
    
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask
    
    print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=120,
            num_beams=6,
            repetition_penalty=1.2,
        )
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
    caption = generated_text.strip()
    print(f"   Generated caption: {caption}")
    
    metadata.append({
        "segment": current_segment,
        "start_time": seg_start_time,
        "end_time": seg_end_time,
        "caption": caption
    })
    current_segment += 1

    # 이후 슬라이딩 윈도우로 세그먼트 처리
    while True:
        count = 0
        # step_frames 만큼 새로운 프레임 추가 (윈도우는 자동으로 오래된 프레임을 제거)
        while count < step_frames:
            try:
                frame = reader.get_data(frame_idx)
                window.append(frame)
                frame_idx += 1
                count += 1
            except Exception:
                break
        if count < step_frames:
            print("Reached end of video during sliding window update.")
            break
        
        seg_start_time = (frame_idx - frames_per_segment) / fps
        seg_end_time = frame_idx / fps
        segment_frames = list(window)
        
        segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
        segment_tensor = segment_tensor.unsqueeze(0)
        segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)
        
        inputs_video = process(processor, video=segment_tensor, text=None)
        pixel_values = inputs_video["pixel_values"].to(device)
        
        inputs_seg = {
            "input_ids": base_input_ids,
            "pixel_values": pixel_values
        }
        video_input_mask_seg = base_video_mask
        
        print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
        with torch.no_grad():
            output = model.generate(
                input_ids=inputs_seg["input_ids"],
                pixel_values=inputs_seg["pixel_values"],
                video_input_mask=video_input_mask_seg,
                max_new_tokens=120,
                num_beams=6,
                repetition_penalty=1.2,
            )
        generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
        caption = generated_text.strip()
        print(f"   Generated caption: {caption}")
        
        metadata.append({
            "segment": current_segment,
            "start_time": seg_start_time,
            "end_time": seg_end_time,
            "caption": caption
        })
        current_segment += 1

total_time = time.time() - start_time_overall
print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

with open("/kaggle/working/video_captions_flan_t5.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)
print("Result saved to '/kaggle/working/video_captions_flan_t5.json'")

Imageio - FPS: 29.97
frames_per_segment: 149, step_frames: 90
Preloading initial window...


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 0: 0.0s - 5.0s
   Generated caption: ji yoon-ah and ji kyung-hee in a park


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 1: 3.0s - 8.0s
   Generated caption: 


...

The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 5: 15.0s - 20.0s
   Generated caption: ji yeon-ah and ji yeon-soo in


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 6: 18.0s - 23.0s
   Generated caption: ji yoon-ah and ji kyung-hee have a romantic dinner together


KeyboardInterrupt: 

In [11]:
!pip install -U bitsandbytes 
!pip install pytorchvideo 
!pip install decord imageio imageio-ffmpeg

import sys
from collections import deque
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image
import json
import time
import re
import imageio

# ==================================================
# 1. 모델 및 프로세서 로드 (EILEV, OPT 기반)
# ==================================================
# 아래 모델 이름은 예시입니다. 실제 사용하시는 OPT 기반 모델 이름으로 수정하세요.
model_name = "kpyu/eilev-blip2-opt-2.7b"  
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

# ==================================================
# 2. 영상 읽기 (imageio를 스트리밍 방식으로 사용)
# ==================================================
video_path = "/kaggle/input/test-video-njz/FuJ1RiLoq-M.mp4"  # 영상 경로 (원하는 영상으로 변경)
try:
    reader = imageio.get_reader(video_path, 'ffmpeg')
    meta = reader.get_meta_data()
    fps = meta['fps']
    print(f"Imageio - FPS: {fps}")
except Exception as e:
    raise Exception(f"Failed to open video with imageio: {e}")

# ==================================================
# 3. 기본 입력 구성 (OPT 기반 EILEV를 위한 단일 서술형 프롬프트)
# ==================================================
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 추가 프롬프트: 동적 행동을 보다 구체적으로 설명하도록 유도
prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

# 기본 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# ==================================================
# 4. 슬라이딩 윈도우를 이용한 세그먼트 분할 및 캡션 생성 (오버랩 적용)
# ==================================================
segment_duration = 5    # 세그먼트 길이 (초)
overlap_duration = 2    # 오버랩 길이 (초)

frames_per_segment = int(fps * segment_duration)
overlap_frames = int(fps * overlap_duration)
step_frames = frames_per_segment - overlap_frames

print(f"frames_per_segment: {frames_per_segment}, step_frames: {step_frames}")

metadata = []
current_segment = 0
start_time_overall = time.time()

# deque를 사용하여 sliding window 구성 (최대 frames_per_segment 개의 프레임 유지)
window = deque(maxlen=frames_per_segment)
frame_idx = 0

# 초기 윈도우 채우기
print("Preloading initial window...")
for i in range(frames_per_segment):
    try:
        frame = reader.get_data(i)
        window.append(frame)
        frame_idx = i + 1
    except Exception:
        break

if len(window) < frames_per_segment:
    print("Not enough frames to form a full segment. Exiting.")
else:
    # 첫 세그먼트 처리 (프레임 0 ~ frames_per_segment-1)
    seg_start_time = 0.0
    seg_end_time = frames_per_segment / fps
    segment_frames = list(window)
    
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)
    
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask
    
    print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=120,
            num_beams=6,
            repetition_penalty=1.2,
        )
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
    caption = generated_text.strip()
    print(f"   Generated caption: {caption}")
    
    metadata.append({
        "segment": current_segment,
        "start_time": seg_start_time,
        "end_time": seg_end_time,
        "caption": caption
    })
    current_segment += 1

    # 이후 슬라이딩 윈도우로 세그먼트 처리
    while True:
        count = 0
        while count < step_frames:
            try:
                frame = reader.get_data(frame_idx)
                window.append(frame)
                frame_idx += 1
                count += 1
            except Exception:
                break
        if count < step_frames:
            print("Reached end of video during sliding window update.")
            break
        
        seg_start_time = (frame_idx - frames_per_segment) / fps
        seg_end_time = frame_idx / fps
        segment_frames = list(window)
        
        segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
        segment_tensor = segment_tensor.unsqueeze(0)
        segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)
        
        inputs_video = process(processor, video=segment_tensor, text=None)
        pixel_values = inputs_video["pixel_values"].to(device)
        
        inputs_seg = {
            "input_ids": base_input_ids,
            "pixel_values": pixel_values
        }
        video_input_mask_seg = base_video_mask
        
        print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
        with torch.no_grad():
            output = model.generate(
                input_ids=inputs_seg["input_ids"],
                pixel_values=inputs_seg["pixel_values"],
                video_input_mask=video_input_mask_seg,
                max_new_tokens=120,
                num_beams=6,
                repetition_penalty=1.2,
            )
        generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
        caption = generated_text.strip()
        print(f"   Generated caption: {caption}")
        
        metadata.append({
            "segment": current_segment,
            "start_time": seg_start_time,
            "end_time": seg_end_time,
            "caption": caption
        })
        current_segment += 1

total_time = time.time() - start_time_overall
print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

with open("/kaggle/working/video_captions_flan_t5.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)
print("Result saved to '/kaggle/working/video_captions_flan_t5.json'")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Imageio - FPS: 29.97
frames_per_segment: 149, step_frames: 90
Preloading initial window...


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 0: 0.0s - 5.0s
   Generated caption: A man and a woman are walking on a bridge. The man is wearing a baseball cap and the woman is wearing a hat. The man is talking to the woman and the woman is looking at the man. The man is looking at the woman and the woman is looking at the man. The man is looking at the woman and the woman is looking at the man. The man is looking at the woman and the woman is looking at the man. The man is looking at the woman and the woman is looking at the man. The man is looking at the woman and the woman is looking at the


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


...

The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 3: 9.0s - 14.0s
   Generated caption: A group of people are playing a game of kick scooter.
Explanation: A group of people are playing a game of kick scooter.
Categories


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 4: 12.0s - 17.0s


KeyboardInterrupt: 

In [1]:
!pip install -U bitsandbytes 
!pip install pytorchvideo 
!pip install decord imageio imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 48.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=4856419f5ca7955b4b67b80861535e76ff0595b240a894a5f62197c13c05efb7
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=9cf4035d15573696a037fe8e0932847285e780c26112ab72b83aec2151915551
  Stored in d

In [3]:

import sys
# 캐글 입력 경로: EILeV 저장소가 /kaggle/input/eilev-git/EILEV-main 에 있다고 가정
sys.path.append("/kaggle/input/eilev-git/EILEV-main")
from collections import deque
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image
import json
import time
import re
import imageio

In [4]:
# ==================================================
# 1. 모델 및 프로세서 로드 (FLAN‑T5 기반 EILEV)
# ==================================================
model_name = "kpyu/eilev-blip2-flan-t5-xl"  # 사용하시는 체크포인트에 맞게 수정
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer


config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/128k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
# ==================================================
# 2. 영상 읽기 (imageio를 스트리밍 방식으로 사용)
# ==================================================
video_path = "/kaggle/input/test-video-njz/FuJ1RiLoq-M.mp4"  # 새 영상 경로

try:
    reader = imageio.get_reader(video_path, 'ffmpeg')
    meta = reader.get_meta_data()
    original_fps = meta['fps']
    print(f"Imageio - Original FPS: {original_fps}")
except Exception as e:
    raise Exception(f"Failed to open video with imageio: {e}")

# 원하는 샘플링 설정: 초당 5프레임
desired_fps = 5  
# 원본 fps에서 몇 프레임마다 샘플링할지 계산 (예: 29.97/5 ≈ 6)
sampling_interval = int(round(original_fps / desired_fps))
print(f"Sampling interval (in original frames): {sampling_interval}")

# ==================================================
# 3. 기본 입력 구성 (EILEV가 학습시킨 프롬프트 사용)
# ==================================================
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼리 토큰 개수

# 기존 EILEV 프롬프트
prompt = "Describe in detail what actions are taking place in the video."
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

# 기본 입력 시퀀스 구성: [bos] + [pad]*num_query_tokens + [newline] + prompt_tokens
base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# ==================================================
# 4. 슬라이딩 윈도우를 이용한 세그먼트 분할 및 캡션 생성
# ==================================================
# 세그먼트 설정: 세그먼트 길이를 6초, 오버랩은 2초로 설정
segment_duration = 6    # 초 단위
overlap_duration = 2    # 초 단위

frames_per_segment = int(desired_fps * segment_duration)  # 예: 5 fps * 6초 = 30 프레임
overlap_frames = int(desired_fps * overlap_duration)        # 예: 5 fps * 2초 = 10 프레임
step_frames = frames_per_segment - overlap_frames           # 예: 30 - 10 = 20 프레임

print(f"frames_per_segment: {frames_per_segment}, step_frames: {step_frames}")

metadata = []
current_segment = 0
start_time_overall = time.time()

# deque를 사용하여 sliding window 구성 (최대 frames_per_segment 개의 프레임 유지)
window = deque(maxlen=frames_per_segment)
sample_index = 0  # 샘플 단위 인덱스 (실제 프레임 번호는 sample_index * sampling_interval)

# 초기 윈도우 채우기 (desired_fps 기반 샘플링)
print("Preloading initial window...")
for i in range(frames_per_segment):
    try:
        frame = reader.get_data(i * sampling_interval)
        window.append(frame)
        sample_index = i + 1
    except Exception:
        break

if len(window) < frames_per_segment:
    print("Not enough frames to form a full segment. Exiting.")
else:
    # 첫 세그먼트 처리 (샘플 0 ~ frames_per_segment-1)
    seg_start_time = 0.0
    seg_end_time = frames_per_segment / desired_fps  # 예: 30 / 5 = 6초
    segment_frames = list(window)
    
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)
    
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask
    
    print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=120,
            num_beams=6,
            repetition_penalty=1.2,
        )
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
    caption = generated_text.strip()
    print(f"   Generated caption: {caption}")
    
    metadata.append({
        "segment": current_segment,
        "start_time": seg_start_time,
        "end_time": seg_end_time,
        "caption": caption
    })
    current_segment += 1

    # 이후 슬라이딩 윈도우로 세그먼트 처리
    while True:
        count = 0
        # step_frames 만큼 새로운 프레임 추가 (윈도우는 자동으로 오래된 프레임 제거)
        while count < step_frames:
            try:
                frame = reader.get_data(sample_index * sampling_interval)
                window.append(frame)
                sample_index += 1
                count += 1
            except Exception:
                break
        if count < step_frames:
            print("Reached end of video during sliding window update.")
            break
        
        seg_start_time = (sample_index - frames_per_segment) / desired_fps
        seg_end_time = sample_index / desired_fps
        segment_frames = list(window)
        
        segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
        segment_tensor = segment_tensor.unsqueeze(0)
        segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)
        
        inputs_video = process(processor, video=segment_tensor, text=None)
        pixel_values = inputs_video["pixel_values"].to(device)
        
        inputs_seg = {
            "input_ids": base_input_ids,
            "pixel_values": pixel_values
        }
        video_input_mask_seg = base_video_mask
        
        print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
        with torch.no_grad():
            output = model.generate(
                input_ids=inputs_seg["input_ids"],
                pixel_values=inputs_seg["pixel_values"],
                video_input_mask=video_input_mask_seg,
                max_new_tokens=120,
                num_beams=6,
                repetition_penalty=1.2,
            )
        generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
        caption = generated_text.strip()
        print(f"   Generated caption: {caption}")
        
        metadata.append({
            "segment": current_segment,
            "start_time": seg_start_time,
            "end_time": seg_end_time,
            "caption": caption
        })
        current_segment += 1

total_time = time.time() - start_time_overall
print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

with open("/kaggle/working/video_captions_flan_t5.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)
print("Result saved to '/kaggle/working/video_captions_flan_t5.json'")

Imageio - Original FPS: 29.97
Sampling interval (in original frames): 6
frames_per_segment: 30, step_frames: 20
Preloading initial window...


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 0: 0.0s - 6.0s
   Generated caption: 


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 1: 4.0s - 10.0s
   Generated caption: 


...

The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 4: 16.0s - 22.0s
   Generated caption: 


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 5: 20.0s - 26.0s


KeyboardInterrupt: 

In [1]:
!pip install -U bitsandbytes 
!pip install pytorchvideo 
!pip install decord imageio imageio-ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 2.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 42.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=9cf53af577c20bad6874380da60bd527dd044ffa355bf51959a597c7c82d0c81
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=4d2e8ad582a9606ac94ada9ea88bad8445124fccf0e68432337f7bc7715d1858
  Stored in d

In [3]:
import sys
# 캐글 입력 경로: EILeV 저장소가 /kaggle/input/eilev-git/EILEV-main 에 있다고 가정
sys.path.append("/kaggle/input/eilev-git/EILEV-main")
from collections import deque
import cv2
import numpy as np
import torch
from transformers import AutoProcessor, BitsAndBytesConfig
from eilev.model.v2 import VideoBlipForConditionalGeneration
from eilev.model.utils import process
from PIL import Image
import json
import time
import re
import imageio

In [4]:
# ==================================================
# 1. 모델 및 프로세서 로드 (EILEV, OPT 기반)
# ==================================================
# 실제 사용하시는 OPT 기반 모델 이름으로 수정하세요.
model_name = "kpyu/eilev-blip2-opt-2.7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

model = VideoBlipForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
)
processor = AutoProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [7]:
# ==================================================
# 2. 영상 읽기 (imageio를 스트리밍 방식으로 사용)
# ==================================================
video_path = "/kaggle/input/test-video-njz/FuJ1RiLoq-M.mp4"  # 영상 경로

try:
    reader = imageio.get_reader(video_path, 'ffmpeg')
    meta = reader.get_meta_data()
    original_fps = meta['fps']
    print(f"Imageio - Original FPS: {original_fps}")
except Exception as e:
    raise Exception(f"Failed to open video with imageio: {e}")

# 원하는 샘플링 설정: 초당 15프레임
desired_fps = 15  
sampling_interval = int(round(original_fps / desired_fps))
print(f"Sampling interval (in original frames): {sampling_interval}")

# ==================================================
# 3. 기본 입력 구성 (OPT 기반 EILEV 프롬프트)
# ==================================================
bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
pad_token = tokenizer.pad_token_id
newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
num_query_tokens = 32  # 영상 관련 쿼스트 토큰 개수

# 기존 프롬프트: 학습 시 사용했던 형식
prompt = " Describe in detail what actions are taking place in the video.:"
prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
num_tokens = base_input_ids.shape[1]

base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# ==================================================
# 후처리 함수: 불필요한 접두어 제거
# ==================================================
def clean_caption(text):
    # 각 줄마다 "Question:", "Answer:", "Explanation:", "Cameraman:" 등 접두어 제거
    lines = text.splitlines()
    filtered = [line.strip() for line in lines if not any(line.strip().startswith(prefix) for prefix in 
                                                           ["Question:", "Answer:", "Explanation:", "Cameraman:"])]
    return " ".join(filtered).strip()

# ==================================================
# 후처리 함수: 중복 문장 제거
# ==================================================
def remove_duplicate_sentences(text):
    # 문장을 마침표를 기준으로 분리
    sentences = re.split(r'\.\s*', text)
    seen = set()
    unique_sentences = []
    for s in sentences:
        s = s.strip()
        if s and s not in seen:
            seen.add(s)
            unique_sentences.append(s)
    # 중복 제거한 문장을 다시 마침표로 이어 붙임
    result = '. '.join(unique_sentences)
    if result and not result.endswith('.'):
        result += '.'
    return result

# ==================================================
# 4. 슬라이딩 윈도우를 이용한 세그먼트 분할 및 캡션 생성 (오버랩 적용)
# ==================================================
# 세그먼트 설정: 세그먼트 길이를 5초, 오버랩은 1초로 설정
segment_duration = 5    # 초 단위
overlap_duration = 1    # 초 단위

frames_per_segment = int(desired_fps * segment_duration)   # 15 fps * 5초 = 75 프레임
overlap_frames = int(desired_fps * overlap_duration)         # 15 fps * 1초 = 15 프레임
step_frames = frames_per_segment - overlap_frames            # 75 - 15 = 60 프레임

print(f"frames_per_segment: {frames_per_segment}, step_frames: {step_frames}")

metadata = []
current_segment = 0
start_time_overall = time.time()

# deque를 사용하여 슬라이딩 윈도우 구성 (최대 frames_per_segment 개의 프레임 유지)
window = deque(maxlen=frames_per_segment)
sample_index = 0  # 실제 프레임 번호 = sample_index * sampling_interval

# 초기 윈도우 채우기 (desired_fps 기반 샘플링)
print("Preloading initial window...")
for i in range(frames_per_segment):
    try:
        frame = reader.get_data(i * sampling_interval)
        window.append(frame)
        sample_index = i + 1
    except Exception:
        break

if len(window) < frames_per_segment:
    print("Not enough frames to form a full segment. Exiting.")
else:
    # 첫 세그먼트 처리 (샘플 인덱스 0 ~ frames_per_segment-1)
    seg_start_time = 0.0
    seg_end_time = frames_per_segment / desired_fps  # 예: 30 / 6 = 5초
    segment_frames = list(window)
    
    segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
    segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
    segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
    inputs_video = process(processor, video=segment_tensor, text=None)
    pixel_values = inputs_video["pixel_values"].to(device)
    
    inputs_seg = {
        "input_ids": base_input_ids,
        "pixel_values": pixel_values
    }
    video_input_mask_seg = base_video_mask
    
    print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
    with torch.no_grad():
        output = model.generate(
            input_ids=inputs_seg["input_ids"],
            pixel_values=inputs_seg["pixel_values"],
            video_input_mask=video_input_mask_seg,
            max_new_tokens=120,
            num_beams=6,
            repetition_penalty=1.2,
        )
    generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
    # 후처리: 불필요한 접두어 제거 후, 중복 문장 제거
    final_caption = remove_duplicate_sentences(clean_caption(generated_text))
    print(f"   Generated caption: {final_caption}")
    
    metadata.append({
        "segment": current_segment,
        "start_time": seg_start_time,
        "end_time": seg_end_time,
        "caption": final_caption
    })
    current_segment += 1

    # 이후 슬라이딩 윈도우로 세그먼트 처리
    while True:
        count = 0
        while count < step_frames:
            try:
                frame = reader.get_data(sample_index * sampling_interval)
                window.append(frame)
                sample_index += 1
                count += 1
            except Exception:
                break
        if count < step_frames:
            print("Reached end of video during sliding window update.")
            break
        
        seg_start_time = (sample_index - frames_per_segment) / desired_fps
        seg_end_time = sample_index / desired_fps
        segment_frames = list(window)
        
        segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
        segment_tensor = segment_tensor.unsqueeze(0)
        segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)
        
        inputs_video = process(processor, video=segment_tensor, text=None)
        pixel_values = inputs_video["pixel_values"].to(device)
        
        inputs_seg = {
            "input_ids": base_input_ids,
            "pixel_values": pixel_values
        }
        video_input_mask_seg = base_video_mask
        
        print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
        with torch.no_grad():
            output = model.generate(
                input_ids=inputs_seg["input_ids"],
                pixel_values=inputs_seg["pixel_values"],
                video_input_mask=video_input_mask_seg,
                max_new_tokens=120,
                num_beams=6,
                repetition_penalty=1.2,
            )
        generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
        final_caption = remove_duplicate_sentences(clean_caption(generated_text))
        print(f"   Generated caption: {final_caption}")
        
        metadata.append({
            "segment": current_segment,
            "start_time": seg_start_time,
            "end_time": seg_end_time,
            "caption": final_caption
        })
        current_segment += 1

total_time = time.time() - start_time_overall
print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# ==================================================
# 5. 결과 출력 및 파일 저장
# ==================================================
for seg in metadata:
    print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

with open("/kaggle/working/video_captions_opt.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4, ensure_ascii=False)
print("Result saved to '/kaggle/working/video_captions_opt.json'")

Imageio - Original FPS: 29.97
Sampling interval (in original frames): 2
frames_per_segment: 75, step_frames: 60
Preloading initial window...


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 0: 0.0s - 5.0s
   Generated caption: A man and a woman are walking on a bridge. The man is wearing a baseball cap and the woman is wearing a hat. The man is talking to the woman. The woman is looking at the man. The man is looking at the woman. The woman is looking at the.


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 1: 4.0s - 9.0s
   Generated caption: A group of people are playing a game of kite flying.


...

Segment 270: 1080.0s - 1085.0s
   Generated caption: Describe in detail what actions are taking place in the video.


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 271: 1084.0s - 1089.0s
   Generated caption: Describe in detail what actions are taking place in the video.


The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


Segment 272: 1088.0s - 1093.0s
   Generated caption: Describe in detail what actions are taking place in the video.
Reached end of video during sliding window update.
Total segmentation and caption generation time: 5877.1s
Segment 0 (0.0s - 5.0s): A man and a woman are walking on a bridge. The man is wearing a baseball cap and the woman is wearing a hat. The man is talking to the woman. The woman is looking at the man. The man is looking at the woman. The woman is looking at the.
Segment 1 (4.0s - 9.0s): A group of people are playing a game of kite flying.
Segment 2 (8.0s - 13.0s): A group of people are playing a game of kick the can.
Segment 3 (12.0s - 17.0s): A group of people are riding a scooter. Categories.
Segment 4 (16.0s - 21.0s): In the video, there is a man and a woman who are talking to each other. The man is wearing a baseball cap and the woman is wearing a hat. The man is holding a fishing rod and the woman is holding a fishing pole. The man is looking at the woman and the

In [4]:
# # ==================================================
# # 1. 모델 및 프로세서 로드 (FLAN‑T5 기반 EILEV)
# # ==================================================
# model_name = "kpyu/eilev-blip2-flan-t5-xl"  # 사용하시는 체크포인트에 맞게 수정
# device = "cuda" if torch.cuda.is_available() else "cpu"
# quant_config = BitsAndBytesConfig(load_in_8bit=True) if device=="cuda" else None

# model = VideoBlipForConditionalGeneration.from_pretrained(
#     model_name,
#     device_map="auto",
#     quantization_config=quant_config,
#     torch_dtype=torch.float16 if device=="cuda" else torch.float32,
# )
# processor = AutoProcessor.from_pretrained(model_name)
# tokenizer = processor.tokenizer

config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/128k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# # ==================================================
# # 2. 영상 읽기 (imageio를 스트리밍 방식으로 사용)
# # ==================================================
# video_path = "/kaggle/input/test-video-njz/FuJ1RiLoq-M.mp4"  # 영상 경로

# try:
#     reader = imageio.get_reader(video_path, 'ffmpeg')
#     meta = reader.get_meta_data()
#     original_fps = meta['fps']
#     print(f"Imageio - Original FPS: {original_fps}")
# except Exception as e:
#     raise Exception(f"Failed to open video with imageio: {e}")

# # 원하는 샘플링 설정: 초당 15프레임
# desired_fps = 15  
# sampling_interval = int(round(original_fps / desired_fps))
# print(f"Sampling interval (in original frames): {sampling_interval}")

# # ==================================================
# # 3. 기본 입력 구성 (OPT 기반 EILEV 프롬프트)
# # ==================================================
# bos_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.pad_token_id
# pad_token = tokenizer.pad_token_id
# newline_token = tokenizer("\n", add_special_tokens=False).input_ids[0]
# num_query_tokens = 32  # 영상 관련 쿼스트 토큰 개수

# # 기존 프롬프트: 학습 시 사용했던 형식
# prompt = "Question: Describe in detail what actions are taking place in the video?\nAnswer:"
# prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids

# base_input_ids = [bos_token] + [pad_token] * num_query_tokens + [newline_token] + prompt_tokens
# base_input_ids = torch.tensor([base_input_ids], dtype=torch.long).to(device)
# num_tokens = base_input_ids.shape[1]

# base_video_mask = [0] + [1] * num_query_tokens + [0] * (num_tokens - (1 + num_query_tokens))
# base_video_mask = torch.tensor([base_video_mask], dtype=torch.bool).to(device)

# # ==================================================
# # 후처리 함수: 불필요한 접두어 제거
# # ==================================================
# def clean_caption(text):
#     # 각 줄마다 "Question:", "Answer:", "Explanation:", "Cameraman:" 등 접두어 제거
#     lines = text.splitlines()
#     filtered = [line.strip() for line in lines if not any(line.strip().startswith(prefix) for prefix in 
#                                                            ["Question:", "Answer:", "Explanation:", "Cameraman:"])]
#     return " ".join(filtered).strip()

# # ==================================================
# # 후처리 함수: 중복 문장 제거
# # ==================================================
# def remove_duplicate_sentences(text):
#     # 문장을 마침표를 기준으로 분리
#     sentences = re.split(r'\.\s*', text)
#     seen = set()
#     unique_sentences = []
#     for s in sentences:
#         s = s.strip()
#         if s and s not in seen:
#             seen.add(s)
#             unique_sentences.append(s)
#     # 중복 제거한 문장을 다시 마침표로 이어 붙임
#     result = '. '.join(unique_sentences)
#     if result and not result.endswith('.'):
#         result += '.'
#     return result

# # ==================================================
# # 4. 슬라이딩 윈도우를 이용한 세그먼트 분할 및 캡션 생성 (오버랩 적용)
# # ==================================================
# # 세그먼트 설정: 세그먼트 길이를 5초, 오버랩은 1초로 설정
# segment_duration = 5    # 초 단위
# overlap_duration = 1    # 초 단위

# frames_per_segment = int(desired_fps * segment_duration)   # 15 fps * 5초 = 75 프레임
# overlap_frames = int(desired_fps * overlap_duration)         # 15 fps * 1초 = 15 프레임
# step_frames = frames_per_segment - overlap_frames            # 75 - 15 = 60 프레임

# print(f"frames_per_segment: {frames_per_segment}, step_frames: {step_frames}")

# metadata = []
# current_segment = 0
# start_time_overall = time.time()

# # deque를 사용하여 슬라이딩 윈도우 구성 (최대 frames_per_segment 개의 프레임 유지)
# window = deque(maxlen=frames_per_segment)
# sample_index = 0  # 실제 프레임 번호 = sample_index * sampling_interval

# # 초기 윈도우 채우기 (desired_fps 기반 샘플링)
# print("Preloading initial window...")
# for i in range(frames_per_segment):
#     try:
#         frame = reader.get_data(i * sampling_interval)
#         window.append(frame)
#         sample_index = i + 1
#     except Exception:
#         break

# if len(window) < frames_per_segment:
#     print("Not enough frames to form a full segment. Exiting.")
# else:
#     # 첫 세그먼트 처리 (샘플 인덱스 0 ~ frames_per_segment-1)
#     seg_start_time = 0.0
#     seg_end_time = frames_per_segment / desired_fps  # 예: 30 / 6 = 5초
#     segment_frames = list(window)
    
#     segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
#     segment_tensor = segment_tensor.unsqueeze(0)  # (1, num_frames, 3, H, W)
#     segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)  # (1, 3, num_frames, H, W)
    
#     inputs_video = process(processor, video=segment_tensor, text=None)
#     pixel_values = inputs_video["pixel_values"].to(device)
    
#     inputs_seg = {
#         "input_ids": base_input_ids,
#         "pixel_values": pixel_values
#     }
#     video_input_mask_seg = base_video_mask
    
#     print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
#     with torch.no_grad():
#         output = model.generate(
#             input_ids=inputs_seg["input_ids"],
#             pixel_values=inputs_seg["pixel_values"],
#             video_input_mask=video_input_mask_seg,
#             max_new_tokens=120,
#             num_beams=6,
#             repetition_penalty=1.2,
#         )
#     generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
#     # 후처리: 불필요한 접두어 제거 후, 중복 문장 제거
#     final_caption = remove_duplicate_sentences(clean_caption(generated_text))
#     print(f"   Generated caption: {final_caption}")
    
#     metadata.append({
#         "segment": current_segment,
#         "start_time": seg_start_time,
#         "end_time": seg_end_time,
#         "caption": final_caption
#     })
#     current_segment += 1

#     # 이후 슬라이딩 윈도우로 세그먼트 처리
#     while True:
#         count = 0
#         while count < step_frames:
#             try:
#                 frame = reader.get_data(sample_index * sampling_interval)
#                 window.append(frame)
#                 sample_index += 1
#                 count += 1
#             except Exception:
#                 break
#         if count < step_frames:
#             print("Reached end of video during sliding window update.")
#             break
        
#         seg_start_time = (sample_index - frames_per_segment) / desired_fps
#         seg_end_time = sample_index / desired_fps
#         segment_frames = list(window)
        
#         segment_tensor = torch.from_numpy(np.stack(segment_frames)).permute(0, 3, 1, 2)
#         segment_tensor = segment_tensor.unsqueeze(0)
#         segment_tensor = segment_tensor.permute(0, 2, 1, 3, 4)
        
#         inputs_video = process(processor, video=segment_tensor, text=None)
#         pixel_values = inputs_video["pixel_values"].to(device)
        
#         inputs_seg = {
#             "input_ids": base_input_ids,
#             "pixel_values": pixel_values
#         }
#         video_input_mask_seg = base_video_mask
        
#         print(f"Segment {current_segment}: {seg_start_time:.1f}s - {seg_end_time:.1f}s")
#         with torch.no_grad():
#             output = model.generate(
#                 input_ids=inputs_seg["input_ids"],
#                 pixel_values=inputs_seg["pixel_values"],
#                 video_input_mask=video_input_mask_seg,
#                 max_new_tokens=120,
#                 num_beams=6,
#                 repetition_penalty=1.2,
#             )
#         generated_text = processor.batch_decode(output, skip_special_tokens=True)[0]
#         final_caption = remove_duplicate_sentences(clean_caption(generated_text))
#         print(f"   Generated caption: {final_caption}")
        
#         metadata.append({
#             "segment": current_segment,
#             "start_time": seg_start_time,
#             "end_time": seg_end_time,
#             "caption": final_caption
#         })
#         current_segment += 1

# total_time = time.time() - start_time_overall
# print(f"Total segmentation and caption generation time: {total_time:.1f}s")

# # ==================================================
# # 5. 결과 출력 및 파일 저장
# # ==================================================
# for seg in metadata:
#     print(f"Segment {seg['segment']} ({seg['start_time']:.1f}s - {seg['end_time']:.1f}s): {seg['caption']}")

# with open("/kaggle/working/video_captions_opt.json", "w", encoding="utf-8") as f:
#     json.dump(metadata, f, indent=4, ensure_ascii=False)
# print("Result saved to '/kaggle/working/video_captions_opt.json'")